In [3]:
import matplotlib.pyplot as plt
import seaborn as sns
import json

import os
os.chdir('..')


from src.swbm import *
from src.plots import *
from src.utils import *

In [4]:
%pwd


'c:\\Users\\motze\\OneDrive\\Master\\ESM\\VegSWBM'

In [5]:
# Load data and format
input_swbm_ger = pd.read_csv('data/Data_swbm_Germany.csv')
input_ger = prepro(input_swbm_ger)

input_swbm_swe = pd.read_csv('data/Data_swbm_Sweden.csv')
input_swe = prepro(input_swbm_swe)

input_swbm_esp = pd.read_csv('data/Data_swbm_Spain.csv')
input_esp = prepro(input_swbm_esp)

sites = [input_ger, input_swe, input_esp]

In [9]:
# Load best parameters
with open('ger_output.json', 'r') as file:
    ger_calib = json.load(file)

with open('swe_output.json', 'r') as file:
    swe_calib = json.load(file)

with open('esp_output.json', 'r') as file:
    esp_calib = json.load(file)

init_params = [ger_calib[0], 
               swe_calib[0], 
               esp_calib[0]]

init_sinus = [ger_calib[3], 
               swe_calib[3], 
               esp_calib[3]]

In [ ]:
# Create test masks
start_test = '2015-01-01'
end_test = '2020-12-31'
train_mask = (input_ger['time'] >= start_test) & (input_ger['time'] <= end_test)